# Description


This notebook showcases locations and basic structure of raw data from:

- S3 (parquet datasets)
- DB (PostGres)

The following exchanges and data types for each universe are included:
**Rewrite the data names according to the protocol https://docs.google.com/document/d/1r53D8XHlDhoSF_rSXfWhXIbX7VR-QeZ_7Y-vnf_nUcQ/edit#heading=h.n1q8t37kb6e4**
- CCXT OHLCV futures realtime (DB)
- CCXT OHLCV futures historical (daily from S3)
- CryptoChassis bid/ask futures historical daily (1 sec)
- CryptoChassis bid/ask futures historical daily (1 min resample)
- CryptoChassis bid/ask spot historical daily (1 sec)
- CryptoChassis bid/ask spot historical daily (1 min resample)

In [ ]:
# TODO(Danya): Replace prints with logs/functions from hprint


# Imports

In [ ]:
import logging
from typing import List, Optional

import pandas as pd

import core.statistics.descriptive as cstadesc
import helpers.hdbg as hdbg
import helpers.henv as henv
import helpers.hparquet as hparque
import helpers.hprint as hprint
import helpers.hsql as hsql
import im_v2.im_lib_tasks as imvimlita

In [ ]:
hdbg.init_logger(verbosity=logging.INFO)

_LOG = logging.getLogger(__name__)

_LOG.info("%s", henv.get_system_signature()[0])

hprint.config_notebook()

# Functions

In [ ]:
# TODO(Danya) -> get_raw_data_from_db
# -> master_raw_data_gallery_lib.py
def get_raw_ccxt_realtime_data(
    db_table: str,
    exchange_id: str,
    start_ts: Optional[str],
    end_ts: Optional[str],
) -> pd.DataFrame:
    """
    Read raw data for given exchange from the RT DB.

    Bypasses the IM Client to avoid any on-the-fly transformations.
    """
    # Get DB connection.
    env_file = imvimlita.get_db_env_path("dev")
    # Connect with the parameters from the env file.
    connection_params = hsql.get_connection_info_from_env_file(env_file)
    connection = hsql.get_connection(*connection_params)
    # Read data from DB.
    query = f"SELECT * FROM {db_table} WHERE exchange_id='{exchange_id}'"
    if start_ts:
        start_ts = pd.Timestamp(start_ts)
        unix_start_timestamp = hdateti.convert_timestamp_to_unix_epoch(start_ts)
        query += f" AND timestamp >='{unix_start_timestamp}'"
    if end_ts:
        end_ts = pd.Timestamp(end_ts)
        unix_end_timestamp = hdateti.convert_timestamp_to_unix_epoch(end_ts)
        query += f" AND timestamp <='{unix_end_timestamp}'"
    rt_data = hsql.execute_query_to_df(connection, query)
    return rt_data

In [ ]:
# -> master_raw_data_gallery_lib.py
def load_parquet_by_period(
    start_ts: pd.Timestamp, end_ts: pd.Timestamp, s3_path: str
) -> pd.DataFrame:
    """
    Read raw historical data from the S3.

    Bypasses the IM Client to avoid any on-the-fly transformations.
    """
    # Create timestamp filters.
    timestamp_filters = hparque.get_parquet_filters_from_timestamp_interval(
        "by_year_month", start_ts, end_ts
    )
    # Load daily data from s3 parquet.
    cc_ba_futures_daily = hparque.from_parquet(
        s3_path, filters=timestamp_filters, aws_profile="ck"
    )
    
    cc_ba_futures_daily = cc_ba_futures_daily.sort_index()
    return cc_ba_futures_daily

In [ ]:
# TODO(Danya): look into the `add` method on a list of Series.
def combine_stats(stats: List[pd.Series]) -> pd.Series:
    """
    Summarize the statistics from pandas Series.

    Example of one statistic Series:

        bid_price    0.0
        bid_size     0.0
        ask_price    0.0
        ask_size     0.0
        dtype: float64
    """
    base_stat = stats[0]
    for stat in stats[1:]:
        base_stat = base_stat.add(stat)
    return base_stat

In [ ]:
# TODO(Danya): Remove, import the function merged into master.
def find_gaps_in_time_series(
    time_series: pd.Series,
    start_timestamp: pd.Timestamp,
    end_timestamp: pd.Timestamp,
    step: str,
) -> pd.Series:
    """
    Find missing points on a time interval specified by [start_timestamp,
    end_timestamp], where point distribution is determined by <step>.

    :param time_series: time series to find gaps in
    :param start_timestamp: start of the time interval to check
    :param end_timestamp: end of the time interval to check
    :param step: distance between two data points on the interval,
      i.e. "S" -> second, "T" -> minute
      for a list of aliases.
    :return: pd.Series representing missing points in the source time series.
    """
    correct_time_series = pd.date_range(
        start=start_timestamp, end=end_timestamp, freq=step
    )
    return correct_time_series.difference(time_series)

In [ ]:
# TODO(Danya): Check if this is the latest version, -> master_raw_data_gallery_lib.py
def process_s3_data_in_chunks(
    start_ts: str, end_ts: str, s3_path: str
) -> List[pd.Series]:
    """
    Load S3 historical data by parts and count the statisticts.
    """
    overall_rows = 0
    gaps_count = 0
    nans_stats = []
    zeros_stats = []
    start_ts = pd.Timestamp(start_ts, tz="UTC")
    end_ts = pd.Timestamp(end_ts, tz="UTC")
    # Separate time period to months.
    # E.g. of `dates` sequence `['2022-10-31 00:00:00+00:00', '2022-11-30 00:00:00+00:00']`.
    dates = pd.date_range(start_ts, end_ts, freq="M")
    for i in range(0, len(dates), 2):
        # Iterate through the dates to select the loading period boundaries.
        # One chunk of data is loaded for two months. 
        # E.g. the sequence of ['2022-09-31', '2022-10-30', '2022-11-31', '2022-12-31']
        # should be divided into two periods: ['2022-09-31', '2022-10-30']
        # and ['2022-11-31', '2022-12-31']
        # TODO(Danya): Pass a month timedelta as a parameter.
        start_end = dates[i : i + 2]
        if len(start_end) == 2:
            start, end = dates[i : i + 2]
            print(f"Loading data for the months {start.month} and {end.month}")
        else:
            start = dates[i]
            end = None
            print(f"Loading data for the month {start.month}")
        # Load the data of the time period.
        daily_data = load_parquet_by_period(start, end, s3_path)
        overall_rows += len(daily_data)
        print("Head:")
        display(daily_data.head(2))
        print("Tail:")
        display(daily_data.tail(2))
        # Count NaNs.
        nans = cstadesc.compute_frac_nan(daily_data)
        # Count zeros.
        zeros = cstadesc.compute_frac_zero(
            daily_data[
                ["bid_price", "bid_size", "ask_price", "ask_size"]
            ]
        )
        nans_stats.append(nans)
        zeros_stats.append(zeros)
        # Count gaps in time series.
        if end is None:
            end = daily_data.index[-1]
        gaps = find_gaps_in_time_series(
            daily_data.index, start, end, "T"
        )
        gaps_count += len(gaps)
    print(f"{overall_rows} rows overall")
    print(f"Found {gaps_count} missing points on a time interval, step - minutes")
    return nans_stats, zeros_stats

# Realtime (the DB data)

## OHLCV

### CCXT futures

In [ ]:
# Get the real time data from DB.
ccxt_rt = get_raw_ccxt_realtime_data(
    "ccxt_ohlcv_futures", "binance", start_ts=None, end_ts=None
)

In [ ]:
print(f"{len(ccxt_rt)} rows overall")
print("Head:")
display(ccxt_rt.head(3))
print("Tail:")
display(ccxt_rt.tail(3))

#### Count NaNs

In [ ]:
# TODO(Danya): Remove this stat
cstadesc.compute_frac_nan(ccxt_rt)

#### Rows with `volume` equal to 0

In [ ]:
# TODO(Danya): Remove this stat
volume0 = ccxt_rt.loc[ccxt_rt["volume"] == 0]
volume0_proc = "{:.2f}".format(len(volume0) * 100 / len(ccxt_rt))
print(
    f"Percentage of data with `volume=0` in real time CCXT data: {volume0_proc}%"
)
print(f"{len(volume0)} overall")
print("First 5 rows:")
display(volume0.head())
print("Last 5 rows:")
display(volume0.tail())

In [ ]:
ax = volume0["currency_pair"].value_counts().plot(kind="bar")
ax = ax.bar_label(ax.containers[-1], label_type="edge")

# Historical (data updated daily)

## OHLCV

### CCXT futures

In [ ]:
s3_path = "s3://cryptokaizen-data/reorg/daily_staged.airflow.pq/ohlcv-futures/ccxt/binance"
# Load daily data from s3 parquet.
ccxt_futures_daily = hparque.from_parquet(s3_path, aws_profile="ck")

In [ ]:
print(f"{len(ccxt_futures_daily)} rows overall")
print("Head:")
display(ccxt_futures_daily.head())
print("Tail:")
display(ccxt_futures_daily.tail())

**Count NaNs**

In [ ]:
# TODO(Danya): Remove this stat

cstadesc.compute_frac_nan(ccxt_futures_daily)

#### Rows with `volume` equal to 0

In [ ]:
# TODO(Danya): Remove this stat

volume0 = ccxt_futures_daily.loc[ccxt_futures_daily["volume"] == 0]
volume0_proc = "{:.2f}".format(len(volume0) * 100 / len(ccxt_futures_daily))
print(
    f"Percentage of data with `volume=0` in historical CCXT data for the period: {volume0_proc}%"
)
print(f"{len(volume0)} overall")
print("First 5 rows:")
display(volume0.head())
print("Last 5 rows:")
display(volume0.tail())

In [ ]:
# TODO(Danya): Remove this stat
ax = volume0["currency_pair"].value_counts().plot(kind="bar")
ax = ax.bar_label(ax.containers[-1], label_type="edge")

## BID-ASK

### CC futures (1 sec)

In [ ]:
s3_path = "s3://cryptokaizen-data/reorg/daily_staged.airflow.pq/bid_ask-futures/crypto_chassis/binance"

The amount of data is too big to process it all at once, so the data will be loaded separately for each month and all statistics will be aggregated.

In [ ]:
start_ts = "20220627-000000"
end_ts = "20221130-000000"
nans_stats, zeros_stats = process_s3_data_in_chunks(start_ts, end_ts, s3_path)

#### Count NaNs

In [ ]:
combine_stats(nans_stats)

#### Count zeros

In [ ]:
combine_stats(zeros_stats)

### CC futures resampled to 1 min

In [ ]:
s3_path = "s3://cryptokaizen-data/reorg/daily_staged.airflow.pq/bid_ask-futures/crypto_chassis.resampled_1min/binance"
# Load daily data from s3 parquet.
cc_ba_futures_resampled = hparque.from_parquet(s3_path, aws_profile="ck")

In [ ]:
print(f"{len(cc_ba_futures_resampled)} rows overall")
print("Head:")
display(cc_ba_futures_resampled.head())
print("Tail:")
display(cc_ba_futures_resampled.tail())

#### Count NaNs

In [ ]:
cstadesc.compute_frac_nan(cc_ba_futures_resampled)

#### Count zeros

In [ ]:
cstadesc.compute_frac_zero(
            cc_ba_futures_resampled[
                ["bid_price", "bid_size", "ask_price", "ask_size"]
            ]
        )

### CC spot (1 sec)

In [ ]:
s3_path = "s3://cryptokaizen-data/reorg/daily_staged.airflow.pq/bid_ask/crypto_chassis/binance"

In [ ]:
start_ts = "20220501-000000"
end_ts = "20221130-000000"
nans_stats, zeros_stats = process_s3_data_in_chunks(start_ts, end_ts, s3_path)

#### Count NaNs

In [ ]:
combine_stats(nans_stats)

#### Count zeros

In [ ]:
combine_stats(zeros_stats)

### CC spot resampled to 1 min

In [ ]:
s3_path = "s3://cryptokaizen-data/reorg/daily_staged.airflow.pq/bid_ask/crypto_chassis.resampled_1min/binance"
# Load daily data from s3 parquet.
cc_ba_spot_resampled = hparque.from_parquet(s3_path, aws_profile="ck")

In [ ]:
print(f"{len(cc_ba_spot_resampled)} rows overall")
print("Head:")
display(cc_ba_spot_resampled.head())
print("Tail:")
display(cc_ba_spot_resampled.tail())

#### Count NaNs

In [ ]:
cstadesc.compute_frac_nan(cc_ba_spot_resampled)

#### Count zeros

In [ ]:
cstadesc.compute_frac_zero(
            cc_ba_spot_resampled[
                ["bid_price", "bid_size", "ask_price", "ask_size"]
            ]
        )